In [1]:
import torch
import pandas as pd

In [2]:
import torch.nn as nn
!pip install transformers
import numpy as np
import pandas as pd
import nltk
from nltk.translate.bleu_score import sentence_bleu
from math import exp
from google.colab import drive
import torch
from torch.utils.data import Dataset, DataLoader
from transformers import GPT2Tokenizer, GPT2LMHeadModel
import json
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import seaborn as sns
import torch
from transformers import GPT2LMHeadModel, GPT2Tokenizer
import random
from collections import defaultdict

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.7/7.7 MB 23.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 28.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 53.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 61.5 MB/s eta 0:00:00


In [3]:
#Mount drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
import json

with open('/content/drive/MyDrive/train-v1.1.json', 'r') as f:
    squad_data = json.load(f)


In [5]:
contexts = []
questions = []
answers = []

for article in squad_data['data']:
    for paragraph in article['paragraphs']:
        context = paragraph['context']
        for qa in paragraph['qas']:
            question = qa['question']
            answer = qa['answers'][0]  # Using the first answer

            contexts.append(context)
            questions.append(question)
            answers.append(answer)


In [6]:
import pandas as pd

df = pd.DataFrame({
    'context': contexts,
    'question': questions,
    'answer_start': [a['answer_start'] for a in answers],
    'answer_text': [a['text'] for a in answers]
})


In [7]:
df_sample = df.sample(frac=0.01, random_state=42)


In [8]:
contexts = df_sample['context'].tolist()
questions = df_sample['question'].tolist()
answers = [{'answer_start': start, 'text': text} for start, text in zip(df_sample['answer_start'], df_sample['answer_text'])]


In [9]:
df_sample.head()

,context,question,answer_start,answer_text
9983,The world's first institution of technology or...,What year was the Banská Akadémia founded?,167,1735
43267,The standard specifies how speed ratings shoul...,What is another speed that can also be reporte...,793,SOS-based speed
81021,The most impressive and famous of Sumerian bui...,Where were the use of advanced materials and t...,421,Sumerian temples and palaces
49374,Ann Arbor has a council-manager form of govern...,Who is elected every even numbered year?,192,mayor
53414,"Shortly before his death, when he was already ...",What was the purpose of top secret ICBM commit...,194,decide on the feasibility of building an ICBM ...


In [10]:
from transformers import BertForQuestionAnswering, BertTokenizerFast

model = BertForQuestionAnswering.from_pretrained('bert-base-uncased')
tokenizer = BertTokenizerFast.from_pretrained('bert-base-uncased')

encoded_data = tokenizer(contexts, questions, truncation=True, padding=True, max_length=512, return_tensors='pt', return_offsets_mapping=True)


Some weights of BertForQuestionAnswering were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['qa_outputs.bias', 'qa_outputs.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [11]:
def char_to_token_offset(context, answer_start_char_pos, answer_text, offsets):
    """Convert character start position to token start position."""
    answer_end_char_pos = answer_start_char_pos + len(answer_text)

    token_start_position = 0
    while token_start_position < len(offsets) and (offsets[token_start_position][0] <= answer_start_char_pos and offsets[token_start_position][1] <= answer_start_char_pos):
        token_start_position += 1

    token_end_position = token_start_position
    while token_end_position < len(offsets) and (offsets[token_end_position][0] <= answer_end_char_pos and offsets[token_end_position][1] <= answer_end_char_pos):
        token_end_position += 1

    token_start_position = min(token_start_position, 511)
    token_end_position = min(token_end_position, 511)

    return token_start_position, token_end_position

start_positions = []
end_positions = []

for i in range(len(contexts)):
    answer_start_char_pos = answers[i]['answer_start']
    answer_text = answers[i]['text']
    offsets = encoded_data['offset_mapping'][i].tolist()

    token_start, token_end = char_to_token_offset(contexts[i], answer_start_char_pos, answer_text, offsets)
    start_positions.append(token_start)
    end_positions.append(token_end)


In [ ]:
from torch.utils.data import DataLoader, TensorDataset
from transformers import AdamW

# Convert start and end positions to tensors
start_positions = torch.tensor(start_positions)
end_positions = torch.tensor(end_positions)

# Create a TensorDataset and DataLoader
dataset = TensorDataset(encoded_data['input_ids'], encoded_data['attention_mask'], start_positions, end_positions)
dataloader = DataLoader(dataset, batch_size=8, shuffle=True)

# Training loop
optimizer = AdamW(model.parameters(), lr=3e-5)

for epoch in range(3):  # Example: Training for 3 epochs
    for batch in dataloader:
        optimizer.zero_grad()
        input_ids = batch[0]
        attention_mask = batch[1]
        start_positions = batch[2]
        end_positions = batch[3]

        outputs = model(input_ids=input_ids, attention_mask=attention_mask, start_positions=start_positions, end_positions=end_positions)
        loss = outputs.loss
        loss.backward()
        optimizer.step()

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
